In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score

# Đọc dữ liệu từ các tệp CSV riêng biệt
df_train = pd.read_csv("dataset/train.csv")
# df_val = pd.read_csv("dataset/validation.csv")
df_test = pd.read_csv("dataset/test.csv")

# Loại bỏ cột 'video_name'
df_train = df_train.drop(columns=["video_name"])
# df_val = df_val.drop(columns=["video_name"])
df_test = df_test.drop(columns=["video_name"])

# Các nhãn bài tập (được đánh số từ 0 đến 5)
label_column = "label"

# Tách feature và label
X_train = df_train.drop(columns=[label_column]).values
y_train = df_train[label_column].values

# X_val = df_val.drop(columns=[label_column]).values
# y_val = df_val[label_column].values

X_test = df_test.drop(columns=[label_column]).values
y_test = df_test[label_column].values

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Train mô hình Logistic Regression
model = LogisticRegression(max_iter=1000, multi_class='ovr')  # One-vs-Rest cho multi-label
model.fit(X_train, y_train)

# Đánh giá trên tập validation
# y_val_pred = model.predict(X_val)
# val_accuracy = accuracy_score(y_val, y_val_pred)
# val_recall = recall_score(y_val, y_val_pred, average='macro')
# print(f'Validation Accuracy: {val_accuracy:.4f}')
# print(f'Validation Recall: {val_recall:.4f}')

# Đếm số lượng mẫu của mỗi lớp
unique, counts = np.unique(y_train, return_counts=True)
class_counts = dict(zip(unique, counts))
print('Class counts:', class_counts)


# Tạo trọng số tùy chỉnh
class_weight = {0: 1/2, 3: 2, 4: 3, 5: 3}  # Giảm trọng số lớp 0, tăng lớp 3, 4, 5
for cls in unique:  
    if cls not in class_weight:  
        class_weight[cls] = 1  # Các lớp khác giữ nguyên trọng số

# Train mô hình với class_weight
model = LogisticRegression(max_iter=1000, multi_class='ovr', class_weight=class_weight)
model.fit(X_train, y_train)

# Đánh giá mô hình
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
recall_per_class = recall_score(y_test, y_pred, average=None)

print(f'Test Accuracy: {accuracy:.4f}')
print(f'Test Recall:')
for i, recall in enumerate(recall_per_class):
    print(f'Class {i}: {recall:.4f}')

d:\Thanh\TLHT\HK6\PBL5\Main\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Class counts: {0: 3150, 1: 2447, 2: 2495, 3: 1316, 4: 927, 5: 805}


d:\Thanh\TLHT\HK6\PBL5\Main\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Test Accuracy: 0.8594
Test Recall:
Class 0: 0.7083
Class 1: 0.7986
Class 2: 1.0000
Class 3: 0.9254
Class 4: 0.9323
Class 5: 0.9890


In [6]:
import pickle


# Lưu mô hình vào file .pkl
with open('Model/Squat_detection_LR.pkl', 'wb') as f:
    pickle.dump(model, f)

with open("Model/scaler_LR.pkl", "wb") as f:
    pickle.dump(scaler, f)
